In [3]:
!pip install -q -U transformers datasets accelerate peft trl bitsandbytes wandb

In [4]:
import os
from dotenv import load_dotenv

# Load environment variables from .env
load_dotenv()

# Access the Hugging Face token
hf_token = os.getenv("hf_token")


In [5]:

import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging,
)

import peft

from peft import LoraConfig
from trl import SFTTrainer

In [6]:
import pandas as pd
file_path = '../../merged.csv'

# Load the CSV file into a pandas DataFrame
df = pd.read_csv(file_path)
df

,Text
0,Civics Unit 1 Grade 11 & 12 hybrid 1Which one ...
1,Civics Unit 1 Grade 11 & 12 hybrid 1 C feature...
2,democracy The president leads the Head of the ...
3,አልሰራ ላላችሁ እዚሁ ስላወረድንላችሁ ስልካችሁ ላይ ጭናችሁ መጠቀም ትችላ...
4,are educated guessHowever prediction must be t...
...,...
61219,አደረሳችሁ አደረሰን! ክህደትን ከሁሉም የከፋ የሚያደርገው ከጠላት የማይመ...
61220,ትወድሃለች ወንድ ልጅ ባንቺ ምክንያት ፈገግ ካለ ይወድሻል ተሰብስበው አሙ...
61221,አሁንም ድረስ ልቤ ይደነግጥልሻል ልክ እንደ መጀመሪያው። የቅዱስ ላሊበላ ...
61222,በተግባሬ መረዳት ትችያለሽ። ለእያንዳንዱ ስሜቴ ቃላቶቼን አውጥቼ እንዲህ ...


In [7]:
dataset=df[['Text']]
dataset

,Text
0,Civics Unit 1 Grade 11 & 12 hybrid 1Which one ...
1,Civics Unit 1 Grade 11 & 12 hybrid 1 C feature...
2,democracy The president leads the Head of the ...
3,አልሰራ ላላችሁ እዚሁ ስላወረድንላችሁ ስልካችሁ ላይ ጭናችሁ መጠቀም ትችላ...
4,are educated guessHowever prediction must be t...
...,...
61219,አደረሳችሁ አደረሰን! ክህደትን ከሁሉም የከፋ የሚያደርገው ከጠላት የማይመ...
61220,ትወድሃለች ወንድ ልጅ ባንቺ ምክንያት ፈገግ ካለ ይወድሻል ተሰብስበው አሙ...
61221,አሁንም ድረስ ልቤ ይደነግጥልሻል ልክ እንደ መጀመሪያው። የቅዱስ ላሊበላ ...
61222,በተግባሬ መረዳት ትችያለሽ። ለእያንዳንዱ ስሜቴ ቃላቶቼን አውጥቼ እንዲህ ...


In [8]:
dataset_2=dataset.copy()

In [7]:
# from datasets import Dataset

# # Create a dictionary containing your Amharic text data
# data_dict = {"Text": dataset['Text'].tolist()}

# # Create a Dataset object
# dataset_2 = Dataset.from_dict(data_dict)
# dataset_2


In [9]:
dataset_2

,Text
0,Civics Unit 1 Grade 11 & 12 hybrid 1Which one ...
1,Civics Unit 1 Grade 11 & 12 hybrid 1 C feature...
2,democracy The president leads the Head of the ...
3,አልሰራ ላላችሁ እዚሁ ስላወረድንላችሁ ስልካችሁ ላይ ጭናችሁ መጠቀም ትችላ...
4,are educated guessHowever prediction must be t...
...,...
61219,አደረሳችሁ አደረሰን! ክህደትን ከሁሉም የከፋ የሚያደርገው ከጠላት የማይመ...
61220,ትወድሃለች ወንድ ልጅ ባንቺ ምክንያት ፈገግ ካለ ይወድሻል ተሰብስበው አሙ...
61221,አሁንም ድረስ ልቤ ይደነግጥልሻል ልክ እንደ መጀመሪያው። የቅዱስ ላሊበላ ...
61222,በተግባሬ መረዳት ትችያለሽ። ለእያንዳንዱ ስሜቴ ቃላቶቼን አውጥቼ እንዲህ ...


In [9]:
!pip install scikit-learn



In [10]:
from sklearn.model_selection import train_test_split


train_val_data, test_data = train_test_split(dataset_2, test_size=0.20, random_state=42)
train_data, evaluation_data = train_test_split(train_val_data, test_size=0.10, random_state=42)

print('Training dataset shape:', len(train_data))
print('evaluation dataset shape:', len(evaluation_data))
print('Testing dataset shape:', len(test_data))

Training dataset shape: 44081
evaluation dataset shape: 4898
Testing dataset shape: 12245


In [11]:
evaluation_data

,Text
47104,እንፈልጋለን ፤ አጨራረሱን እንፈልጋለን ኦባ የስኬታችን አንዱ አካል ነው።...
18751,ተችሏል ተብሏል። በሌላ ዜና በአሸባሪው ሸኔ አባላት መካከል የተፈጠረው አ...
38337,ያጋልጠዋል። ለ39 አመታት ኮማ ውስጥ ከቆየ በኋላ ዛሬ በ73 አመቱ ከዚህ...
1973,ሹመቶችን ካገኘው ከተቃዋሚው ብሄራዊ ባይቶና አባይ ትግራይ ፓርቲ እንደኾነ...
37065,አስተማማኝ ዝውውር አደረገ ። || ማን ሲቲ በቀጣዩ ክረምት ሜሲን ለማስፈ...
...,...
10796,ዋና አሰልጣኝ)፡ በጣም የተወዳደርን ይመስለኛል፣ እዚህ በጣም ደስተኛ ነኝ...
11007,እንደማይዝ በማወቁ ነው ጥፋቱን የፈፀመው ሮድሪጎ ያችን ኳስ በፍጥነት ባሄ...
14322,ተንቀሳቅሰዋል። ቡድኑ በቅድሚያ በጅግጅጋ ከተማ ቅዳሜ ሚያዝያ 29 ቀን 2...
36493,አሜሪካዊ አርቲስት 1 Photo Instagram ላይ በለቀቀች ቁጥር 12 ...


In [11]:
import numpy as np
#Divide the dataset into train and test categories 
msk = np.random.rand(len(dataset_2)) < 0.8
train_dataset = dataset_2[msk]
test_dataset = dataset_2[~msk]

In [12]:
#convert format of the dataset to HuggingFace Dataset from Pandas DataFrame
from datasets import Dataset

test_dataset=Dataset.from_pandas(test_dataset)

In [13]:

#convert the format of the dataset to HuggingFace Dataset from Pandas DataFrame
train_dataset=Dataset.from_pandas(train_dataset)

In [14]:

#convert the format of the dataset to HuggingFace Dataset from Pandas DataFrame
evaluation_dataset=Dataset.from_pandas(evaluation_data)

In [15]:
test_dataset

Dataset({
    features: ['Text', '__index_level_0__'],
    num_rows: 12404
})

In [16]:

#remove unnecessary column
test_dataset=test_dataset.remove_columns("__index_level_0__")
train_dataset=train_dataset.remove_columns("__index_level_0__")
evaluation_dataset=evaluation_dataset.remove_columns("__index_level_0__")

In [17]:

import datasets
#combine the train and test dataset into one datset
main_dataset= datasets.DatasetDict({
    'train': train_dataset,
    'test': test_dataset,
    'evaluate': evaluation_dataset
})

In [18]:
main_dataset

DatasetDict({
    train: Dataset({
        features: ['Text'],
        num_rows: 48820
    })
    test: Dataset({
        features: ['Text'],
        num_rows: 12404
    })
    evaluate: Dataset({
        features: ['Text'],
        num_rows: 4898
    })
})

Model

In [19]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    AutoTokenizer,
    TrainingArguments,
    pipeline,
)
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training
from trl import SFTTrainer

In [20]:
# Model
base_model = "NousResearch/Llama-2-7b-hf"
new_model = "llama-2-7b-Amh"


# Tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model, use_fast=True)
tokenizer.pad_token = tokenizer.unk_token
tokenizer.padding_side = "right"

In [21]:
# Quantization configuration
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

# LoRA configuration
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=['up_proj', 'down_proj', 'gate_proj', 'k_proj', 'q_proj', 'v_proj', 'o_proj']
)


In [22]:
def load_model(model_name, bnb_config):
    n_gpus = torch.cuda.device_count()
    max_memory = f'{23000}MB'

load_model(base_model,bnb_config)

In [23]:
import torch
torch.cuda.empty_cache()


In [24]:
# Load base moodel
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=bnb_config,
    device_map={"": 0}
)



Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/home/elias_assamnew/.conda/envs/venv/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:393: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/home/elias_assamnew/.conda/envs/venv/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:398: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


In [25]:
# Cast the layernorm in fp32, make output embedding layer require grads, add the upcasting of the lmhead to fp32
model = prepare_model_for_kbit_training(model)

In [26]:
training_dataset=main_dataset

In [27]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear4bit(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )


In [28]:
import torch

# Use the appropriate device (cuda:0, cuda:1, etc.)
device = torch.device("cuda:0")

# Empty the cache
torch.cuda.empty_cache()

# Print GPU memory usage
print(torch.cuda.memory_summary(device))


|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |   4273 MiB |   4523 MiB |  17541 MiB |  13267 MiB |
|       from large pool |   4110 MiB |   4360 MiB |  17247 MiB |  13137 MiB |
|       from small pool |    163 MiB |    163 MiB |    293 MiB |    130 MiB |
|---------------------------------------------------------------------------|
| Active memory         |   4273 MiB |   4523 MiB |  17541 MiB |  13267 MiB |
|       from large pool |   4110 MiB |   4360 MiB |  17247 MiB |

In [29]:
import torch
torch.cuda.empty_cache()


In [ ]:
import torch

# Check the number of available GPUs
n_gpus = torch.cuda.device_count()
print(f"Number of available GPUs: {n_gpus}")

# Check the memory of each available GPU
for i in range(n_gpus):
    gpu_memory = torch.cuda.get_device_properties(i).total_memory
    print(f"GPU {i}: Total memory: {gpu_memory / (1024**3)} GB")


Number of available GPUs: 1
GPU 0: Total memory: 22.01849365234375 GB


In [30]:
from transformers import Trainer, TrainingArguments, BitsAndBytesConfig


In [31]:
# Set training arguments
training_arguments = TrainingArguments(
        output_dir="../results",
        num_train_epochs=1,
        per_device_train_batch_size=10,
        per_device_eval_batch_size=1,
        gradient_accumulation_steps=1,
        gradient_checkpointing=True,
        fp16=True,
        evaluation_strategy="steps",
        eval_steps=1000,
        logging_steps=1,
        optim="paged_adamw_8bit",
        learning_rate=2e-4,
        lr_scheduler_type="linear",
        warmup_steps=10,
        # report_to="wandb",
        max_steps=10, # Remove this line for a real fine-tuning
)

# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=main_dataset["train"],
    eval_dataset=main_dataset["evaluate"],
    peft_config=peft_config,
    dataset_text_field="Text",
    max_seq_length=512,
    tokenizer=tokenizer,
    args=training_arguments,
)
# data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False)
model.config.use_cache = False  # re-enable for inference to speed up predictions for similar inputs

# Train model
trainer.train()

# Save trained model
trainer.model.save_pretrained(new_model)


Map:   0%|          | 0/48820 [00:00<?, ? examples/s]

Map:   0%|          | 0/4898 [00:00<?, ? examples/s]

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: elias-assamnew. Use `wandb login --relogin` to force relogin


/home/elias_assamnew/.conda/envs/venv/lib/python3.11/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss,Validation Loss


Inference

In [32]:
# Run text generation pipeline with our model
prompt = "የኢትዮጽያ  ጂዲፒ ምን ያህል ነበር"
instruction = f"### Instruction:\n{prompt}\n\n### Response:\n"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=128)
result = pipe(instruction)
print(result[0]['generated_text'][len(instruction):])

/home/elias_assamnew/.conda/envs/venv/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:393: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/elias_assamnew/.conda/envs/venv/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:398: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/home/elias_assamnew/.conda/envs/venv/lib/python3.11/site-packages/torch/utils/checkpoint.py:61: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(



የኢትዮጽያ ጂዲፒ አስተዋይን አለም

##


In [33]:
# Run text generation pipeline with our model
prompt = "how can i treat flu, give the response in amharic language"
instruction = f"### Instruction:\n{prompt}\n\n### Response:\n"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=128)
result = pipe(instruction)
print(result[0]['generated_text'][len(instruction):])


በአማርኛ በማለት በመጀመሪያ መጀመሪያ በመጀመሪያ በመጀመሪያ


In [34]:
# Run text generation pipeline with our model
prompt = "tell me about ethiopian politics, give the response in amharic language"
instruction = f"### Instruction:\n{prompt}\n\n### Response:\n"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=128)
result = pipe(instruction)
print(result[0]['generated_text'][len(instruction):])


<p>አስተዋጭ አስተዋጭ አስተዋጭ አስተዋጭ አስተዋጭ አስተዋጭ


In [35]:
# Run text generation pipeline with our model
prompt = "who the prime minister of ethiopia, give the response in amharic language"
instruction = f"### Instruction:\n{prompt}\n\n### Response:\n"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=128)
result = pipe(instruction)
print(result[0]['generated_text'][len(instruction):])


<p>አስተዳደር አለም አለም አለም አለም አለም አለም አለም አለ


In [36]:
# Run text generation pipeline with our model
prompt = "3 Ethiopian premier league club, give the response in amharic language"
instruction = f"### Instruction:\n{prompt}\n\n### Response:\n"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=128)
result = pipe(instruction)
print(result[0]['generated_text'][len(instruction):])


<p>በአማርኛ በማለት የአማርኛ ተራራ መግለጫ በአማርኛ ተራራ ��


In [37]:
# Empty VRAM
del model
del pipe
del trainer
import gc
gc.collect()
gc.collect()

0

In [40]:
import torch
torch.cuda.empty_cache()

## Merge the base model with the trained Adapters

In [39]:
# Reload model in FP16 and merge it with LoRA weights
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map={"": 0},
)
model = PeftModel.from_pretrained(model, new_model)
model = model.merge_and_unload()

# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 86.00 MiB. GPU 0 has a total capacty of 22.02 GiB of which 2.25 MiB is free. Process 31451 has 5.34 GiB memory in use. Including non-PyTorch memory, this process has 16.66 GiB memory in use. Of the allocated memory 15.47 GiB is allocated by PyTorch, and 159.02 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF